<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bdd02f7feeb489ab7f997ad671420693b86bb55015d4c23a368b70183e6e3cd7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-04 11:47:20
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: 16.09 K (0.11%)
Current PnL: -24.35 L (-15.73%)
CY Booked + Current PnL: -10.06 L (-6.5%)
-------------------
Total profit:  1.34 L
Total loss:  -25.68 L
-------------------
Total Booked + Current PnL: 16.92 L (13.24%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.93%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 88.98 L (61.82%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.07,-7.70,8.37,0.02,13485.0,-13446.0,161113.0,145.54,68.0,M-SC,15.57,234.0,-1.00,1.16,40.14,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.12,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.60,10.15,OX40N,NTT,DURABLES
43,ITC,452.00,0.54,-1.62,12.26,10.44,24139.0,-3242.0,196896.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.85,X40,NTT,FMCG
50,MASFIN,398.61,-2.16,-4.12,27.29,22.05,25638.0,-4035.0,93945.0,-17.34,52.0,H-SC,6.90,164.0,-0.16,0.68,36.56,XR,ATH,FINANCE
35,ICICIGI,2252.93,0.15,5.39,14.05,20.19,25812.0,9392.0,183712.0,-17.43,46.0,X-MC,6.60,68.0,0.36,1.32,21.32,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,4.99,-44.28,212.57,74.15,193766.0,-72451.0,91154.0,-264.07,43.0,H-SC,19.99,149.0,-0.37,0.66,16.84,XR,NTT,CHEMICALS
29,HAPPSTMNDS,1480.71,3.06,-36.04,189.00,84.85,154935.0,-46189.0,81976.0,-23.94,57.0,H-SC,13.99,147.0,-0.30,0.59,5.21,AR,ATH,IT
40,INDUSINDBK,1800.00,2.37,-36.00,107.62,32.89,53181.0,-27790.0,49416.0,-710.16,65.0,L-MC,6.23,259.0,-0.52,0.36,36.09,XR,NTT,BANKS
15,CAMS,4762.00,2.11,3.27,20.84,24.79,54205.0,8243.0,260099.0,-1.47,54.0,X-SC,2.14,86.0,0.15,1.87,27.17,X40N,NTT,MISC
73,TCS,4389.97,1.88,-11.97,35.51,19.28,108139.0,-41424.0,304532.0,-23.30,70.0,X-LC,6.53,1.0,-0.38,2.19,12.58,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-4.76,-56.90,238.03,45.70,181403.0,-100605.0,76210.0,-20.98,52.0,M-SC,23.54,205.0,-0.55,0.55,9.85,XY24,NTT,TRAVEL
44,JCHAC,2282.00,-3.45,-33.74,50.92,-0.01,43118.0,-43127.0,84678.0,17226.09,8.0,M-SC,0.97,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
50,MASFIN,398.61,-2.16,-4.12,27.29,22.05,25638.0,-4035.0,93945.0,-17.34,52.0,H-SC,6.90,164.0,-0.16,0.68,36.56,XR,ATH,FINANCE
51,MEDANTA,1486.00,-2.04,-0.83,25.12,24.08,31625.0,-1054.0,125896.0,-8.86,36.0,X-SC,5.96,89.0,-0.03,0.91,18.62,XY24,NTT,HEALTHCARE
28,GREENPANEL,537.00,-1.73,-35.86,123.18,43.14,144635.0,-65660.0,117418.0,162.92,28.0,M-SC,7.14,240.0,-0.45,0.84,10.45,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.62,-0.52,107.8,106.72,132265.0,-641.0,122695.0,-50.82,38.0,H-SC,1.57,139.0,-0.0,0.88,22.74,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.57,-5.51,120.05,107.91,167472.0,-8142.0,139502.0,-22.98,36.0,M-SC,11.28,216.0,-0.05,1.00,0.57,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.07,-7.70,8.37,0.02,13485.0,-13446.0,161113.0,145.54,68.0,M-SC,15.57,234.0,-1.00,1.16,40.14,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.12,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.60,10.15,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.38,-23.42,48.18,13.47,99494.0,-63162.0,206505.0,-68.51,32.0,H-SC,2.37,158.0,-0.63,1.49,5.00,XY24,NTT,PAINTS
66,SIS,528.00,-0.55,-24.88,61.42,21.26,51230.0,-27622.0,83410.0,1966.84,45.0,H-SC,4.08,166.0,-0.54,0.60,12.70,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,1.01,5.52,63.26,72.27,113106.0,9348.0,178796.0,-9.26,70.0,M-LC,3.07,99.0,0.08,1.29,12.03,XR,NTT,IT
37,IEX,219.0,0.38,1.43,46.54,48.63,94385.0,2850.0,202804.0,-32.45,67.0,H-SC,14.71,136.0,0.03,1.46,13.14,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.01,5.52,63.26,72.27,113106.0,9348.0,178796.0,-9.26,70.0,M-LC,3.07,99.0,0.08,1.29,12.03,XR,NTT,IT
37,IEX,219.00,0.38,1.43,46.54,48.63,94385.0,2850.0,202804.0,-32.45,67.0,H-SC,14.71,136.0,0.03,1.46,13.14,XR,NTT,MISC
38,INDIAMART,4810.62,-0.62,-0.52,107.80,106.72,132265.0,-641.0,122695.0,-50.82,38.0,H-SC,1.57,139.0,-0.00,0.88,22.74,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.13,-1.28,37.38,35.63,77339.0,-2674.0,206900.0,-15.41,43.0,H-MC,3.41,119.0,-0.03,1.49,15.18,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.57,-5.51,120.05,107.91,167472.0,-8142.0,139502.0,-22.98,36.0,M-SC,11.28,216.0,-0.05,1.00,0.57,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.05,-20.69,87.17,48.45,138627.0,-41475.0,159031.0,-26.32,13.0,X-MC,8.85,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.39,-37.20,115.28,35.20,93157.0,-47861.0,80809.0,5.99,25.0,X-SC,14.37,92.0,-0.51,0.58,0.92,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.73,-22.22,97.36,53.50,228495.0,-67060.0,234691.0,-62.35,26.0,X-MC,1.48,58.0,-0.29,1.69,4.24,XY24,NTT,FMCG
22,DIXON,18931.72,-1.16,-11.01,36.63,21.58,45679.0,-15435.0,124704.0,-55.16,26.0,X-MC,5.72,56.0,-0.34,0.90,10.40,X40N,ATH,IT
46,JSWINFRA,342.00,-0.74,-9.66,27.54,15.23,49848.0,-19346.0,181001.0,-22.76,30.0,X-MC,6.80,66.0,-0.39,1.30,19.48,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.05,-23.48,114.52,64.15,208518.0,-55871.0,182080.0,-55.44,42.0,X-SC,1.48,82.0,-0.27,1.31,2.14,XY24,BTT,CEMENT
8,AWL,485.0,-0.73,-22.22,97.36,53.50,228495.0,-67060.0,234691.0,-62.35,26.0,X-MC,1.48,58.0,-0.29,1.69,4.24,XY24,NTT,FMCG
43,ITC,452.0,0.54,-1.62,12.26,10.44,24139.0,-3242.0,196896.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.85,X40,NTT,FMCG
15,CAMS,4762.0,2.11,3.27,20.84,24.79,54205.0,8243.0,260099.0,-1.47,54.0,X-SC,2.14,86.0,0.15,1.87,27.17,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.04,1.38,29.19,30.97,52300.0,2432.0,179170.0,-22.52,48.0,X-MC,2.21,75.0,0.05,1.29,14.11,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.05,-20.69,87.17,48.45,138627.0,-41475.0,159031.0,-26.32,13.0,X-MC,8.85,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
55,QUESS,424.00,-0.85,-30.71,106.16,42.84,47817.0,-19964.0,45042.0,-54.82,36.0,X-SC,37.31,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
19,COLPAL,3726.84,0.60,-19.71,76.25,41.51,161231.0,-51915.0,211450.0,-4.32,34.0,X-MC,7.37,61.0,-0.32,1.52,0.60,XY25,ATH,FMCG
53,PAGEIND,51605.08,0.70,-8.39,37.61,26.07,56415.0,-13740.0,150000.0,-32.53,32.0,X-MC,10.70,55.0,-0.24,1.08,0.70,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.39,-37.20,115.28,35.20,93157.0,-47861.0,80809.0,5.99,25.0,X-SC,14.37,92.0,-0.51,0.58,0.92,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.85,-30.71,106.16,42.84,47817.0,-19964.0,45042.0,-54.82,36.0,X-SC,37.31,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,0.10,-47.35,189.26,52.31,144653.0,-68729.0,76431.0,-43.96,36.0,X-SC,5.36,91.0,-0.48,0.55,2.31,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.39,-37.20,115.28,35.20,93157.0,-47861.0,80809.0,5.99,25.0,X-SC,14.37,92.0,-0.51,0.58,0.92,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.16,-11.01,36.63,21.58,45679.0,-15435.0,124704.0,-55.16,26.0,X-MC,5.72,56.0,-0.34,0.90,10.40,X40N,ATH,IT
51,MEDANTA,1486.00,-2.04,-0.83,25.12,24.08,31625.0,-1054.0,125896.0,-8.86,36.0,X-SC,5.96,89.0,-0.03,0.91,18.62,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.88,-11.97,35.51,19.28,108139.0,-41424.0,304532.0,-23.30,70.0,X-LC,6.53,1.0,-0.38,2.19,12.58,X40,ATH,IT
41,INFY,2275.00,0.94,8.48,42.77,54.87,145168.0,26528.0,339416.0,-15.65,67.0,X-LC,2.98,2.0,0.18,2.44,17.46,X40,BTT,IT
75,TMPV,600.00,-0.27,-15.66,68.59,42.18,159893.0,-43296.0,233114.0,-24.30,31.0,X-LC,3.55,3.0,-0.27,1.68,0.98,XY24,NTT,AUTO
81,VBL,671.64,1.63,-1.97,38.40,35.67,118894.0,-6221.0,309621.0,-13.58,62.0,X-LC,5.28,4.0,-0.05,2.23,11.61,X40N,ATH,FMCG
43,ITC,452.00,0.54,-1.62,12.26,10.44,24139.0,-3242.0,196896.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.85,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.37,-36.00,107.62,32.89,53181.0,-27790.0,49416.0,-710.16,65.0,L-MC,6.23,259.0,-0.52,0.36,36.09,XR,NTT,BANKS
6,ASIANTILES,137.00,0.83,-14.37,112.17,81.67,90101.0,-13485.0,80325.0,7222.22,47.0,L-SC,16.46,271.0,-0.15,0.58,55.55,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.16,-4.12,27.29,22.05,25638.0,-4035.0,93945.0,-17.34,52.0,H-SC,6.90,164.0,-0.16,0.68,36.56,XR,ATH,FINANCE
13,BSOFT,831.70,1.83,-21.95,93.19,50.79,102302.0,-30875.0,109778.0,0.12,74.0,H-SC,5.57,151.0,-0.30,0.79,27.82,XR,ATH,IT
69,SURYODAY,216.00,-1.06,-20.59,54.94,23.03,78124.0,-36873.0,142198.0,55.70,46.0,H-SC,10.07,167.0,-0.47,1.02,41.09,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.07,-7.70,8.37,0.02,13485.0,-13446.0,161113.0,145.54,68.0,M-SC,15.57,234.0,-1.00,1.16,40.14,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.62,3.87,16.44,20.95,42992.0,9742.0,261510.0,2.88,80.0,X-LC,17.96,31.0,0.23,1.88,40.88,X40,ATH,PAINTS
13,BSOFT,831.70,1.83,-21.95,93.19,50.79,102302.0,-30875.0,109778.0,0.12,74.0,H-SC,5.57,151.0,-0.30,0.79,27.82,XR,ATH,IT
40,INDUSINDBK,1800.00,2.37,-36.00,107.62,32.89,53181.0,-27790.0,49416.0,-710.16,65.0,L-MC,6.23,259.0,-0.52,0.36,36.09,XR,NTT,BANKS
32,HCLTECH,1922.01,1.49,8.05,15.44,24.73,40361.0,19481.0,261405.0,12.76,68.0,X-LC,5.22,7.0,0.48,1.88,24.51,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.43
1,25,45.25
2,50,76.21


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.24
MC    29.46
LC    24.33
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.95
X40      21.91
X40N     12.17
XR        9.99
AR        9.27
XY25      9.20
OX40N     7.70
SR        1.03
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.42
X-MC    22.86
X-LC    20.84
M-SC    12.05
X-SC     8.35
H-MC     4.84
L-SC     1.42
M-MC     1.40
M-LC     1.29
H-LC     1.20
L-LC     1.00
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.06,-6.18,40.51
IT,12.55,-15.56,78.04
FINANCE,9.47,-17.81,67.59
MISC,7.61,-22.73,73.01
PAINTS,6.15,-8.10,24.16
ELECTRICAL,6.14,-10.06,50.38
INSURANCE,4.44,-2.53,38.66
PHARMA,3.98,-0.87,33.15
AUTO,3.41,-21.50,73.60


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3154950.0,21
AR,1284792.0,10
XR,1277389.0,13
X40,991680.0,14
X40N,756623.0,9
OX40N,721582.0,10
XY25,361947.0,6
SR,276805.0,2
MH,71969.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3588463.0,25
M-SC,1383036.0,15
X-MC,1304227.0,16
X-LC,866323.0,11
X-SC,758864.0,8
H-MC,407116.0,3
L-SC,264528.0,3
M-LC,113106.0,1
H-LC,67388.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1257572.0      6
           AR         881287.0      5
M-SC       XY24       792085.0      6
H-SC       XR         765981.0      7
X-MC       X40        451805.0      7
           XY24       389850.0      3
X-LC       X40        360799.0      5
H-SC       OX40N      334849.0      4
M-SC       OX40N      299663.0      5
X-SC       X40N       291828.0      3
           XY24       287960.0      3
H-SC       SR         276805.0      2
X-MC       X40N       272312.0      4
X-LC       XY24       232859.0      2
H-MC       AR         212492.0      2
           XY24       194624.0      1
X-LC       X40N       192483.0      2
X-MC       XY25       190260.0      2
X-SC       X40        179076.0      2
L-SC       XR         177458.0      2
M-SC       XR         167663.0      2
           AR         123625.0      2
M-LC       XR         113106.0      1
L-SC       OX40N       87070.0      1
X-LC       XY25        80182.0      2
H-SC       MH          71969.0      1
H-LC       AR          67388.0      1
L-MC       XR          53181.0      1
M-MC       XY25        51297.0      1
L-LC       XY25        40208.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
